## Evaluating the models with the Test Set 

In [1]:
import sys 
sys.path.append('../')
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

import time
import timeit
from NN import activation_functions, loss_functions
import NN.network as network

from utils import dataset_helper
from utils import custom_scores
from importlib import reload 


base_dir = '../data/fashion'
print(os.listdir(base_dir))

['train-labels-idx1-ubyte.gz', 'train-images-idx3-ubyte.gz', 't10k-images-idx3-ubyte.gz', 't10k-labels-idx1-ubyte.gz']


/root/.local/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
from utils import mnist_reader, dataset_helper
X, y = mnist_reader.load_mnist('../data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('../data/fashion', kind='t10k')

In [3]:
X = X.copy() / 255.

Given the dtypes, there is no possibility of negative values in the dataset. 

In [4]:
nclasses =10
X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.05, random_state=42)    
y_train = dataset_helper.one_hot_encode(y_train, nclasses)
y_val = dataset_helper.one_hot_encode(y_val, nclasses)

In [5]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((57000, 784), (3000, 784), (57000, 10), (3000, 10))

In [6]:
lr = 0.0001
b_sz = 256
eps = np.finfo(np.float64).eps
nfeatures  = X_train.shape[1]
epoch_sz = X_train.shape[0]
max_iter = 1000 * (epoch_sz // b_sz) 
print_interval = 10 * (epoch_sz // b_sz) 
decay_iteractions= 300 * (epoch_sz // b_sz) 
decay_rate = 0.5

In [ ]:
reload(custom_scores)
reload(dataset_helper)
reload(loss_functions)
reload(activation_functions)
reload(network)
reload(dataset_helper)


h1 = network.Layer(nfeatures, 128, 'relu',  label="H1")
h2 = network.Layer(128, 256, 'relu',  label="H2")
h3 = network.Layer(256, 100, 'relu',  label="H3")
o1 = network.Layer(100, nclasses, 'softmax', label="soft")

model = network.NN(loss='cross_entropy')
model.add_layer(h1)
model.add_layer(h2)
model.add_layer(h3)
model.add_layer(o1)
model.summary()

print("")
model.fit(X_train, y_train, max_iter=max_iter, 
          lr=lr, epsilon=eps, b_sz = b_sz,
          X_val=X_val, Y_val=y_val,
          decay_iteractions= decay_iteractions, decay_rate = decay_rate,
          print_interval=print_interval)
iteraction_log = network.get_iteration_log()

Y_ = np.array(model.predict(X_val)).argmax(axis=-1)
reload(custom_scores)
custom_scores.evaluate_multiclass(y_val=y_val.argmax(axis=-1), y_pred=Y_)
iteraction_log.index = iteraction_log.it
iteraction_log.error_train.plot()
iteraction_log.error_val.plot()

Model Summary
-------------------------------
H1      (input=784, neurons=128, activation=relu)
H2      (input=128, neurons=256, activation=relu)
H3      (input=256, neurons=100, activation=relu)
soft    (input=100, neurons=10, activation=softmax)
-------------------------------

Shuffled
It: 2220 Batch: 213 Epoch 9 Train Loss: 0.09767297 lr: 0.000100 Val Loss: 0.07559946 Val Acc 0.85100000
It: 4440 Batch: 203 Epoch 19 Train Loss: 0.06763070 lr: 0.000100 Val Loss: 0.06760438 Val Acc 0.86400000
It: 6660 Batch: 193 Epoch 29 Train Loss: 0.05983446 lr: 0.000100 Val Loss: 0.06252444 Val Acc 0.87433333
It: 8880 Batch: 183 Epoch 39 Train Loss: 0.05450625 lr: 0.000100 Val Loss: 0.05914234 Val Acc 0.87933333
It: 11100 Batch: 173 Epoch 49 Train Loss: 0.05053760 lr: 0.000100 Val Loss: 0.06079744 Val Acc 0.88066667
It: 13320 Batch: 163 Epoch 59 Train Loss: 0.04700024 lr: 0.000100 Val Loss: 0.05775611 Val Acc 0.88833333
It: 15540 Batch: 153 Epoch 69 Train Loss: 0.04398248 lr: 0.000100 Val Loss: 0.0

It: 168720 Batch: 132 Epoch 756 Train Loss: 0.00030367 lr: 0.000025 Val Loss: 0.15322101 Val Acc 0.88633333
It: 170940 Batch: 122 Epoch 766 Train Loss: 0.00029937 lr: 0.000025 Val Loss: 0.15360567 Val Acc 0.88733333
It: 173160 Batch: 112 Epoch 776 Train Loss: 0.00029405 lr: 0.000025 Val Loss: 0.15369592 Val Acc 0.88666667
It: 175380 Batch: 102 Epoch 786 Train Loss: 0.00028864 lr: 0.000025 Val Loss: 0.15430726 Val Acc 0.88666667
It: 177600 Batch: 92 Epoch 796 Train Loss: 0.00028437 lr: 0.000025 Val Loss: 0.15457418 Val Acc 0.88700000
It: 179820 Batch: 82 Epoch 806 Train Loss: 0.00027972 lr: 0.000025 Val Loss: 0.15502829 Val Acc 0.88600000
It: 182040 Batch: 72 Epoch 816 Train Loss: 0.00027563 lr: 0.000025 Val Loss: 0.15536553 Val Acc 0.88666667
It: 184260 Batch: 62 Epoch 826 Train Loss: 0.00027062 lr: 0.000025 Val Loss: 0.15587903 Val Acc 0.88766667
It: 186480 Batch: 52 Epoch 836 Train Loss: 0.00026637 lr: 0.000025 Val Loss: 0.15637974 Val Acc 0.88700000


In [ ]:
reload(custom_scores)
reload(dataset_helper)
reload(loss_functions)
reload(activation_functions)
reload(network)
reload(dataset_helper)


h1 = network.Layer(nfeatures, 512, 'relu',  label="H1")
o1 = network.Layer(512, nclasses, 'softmax', label="soft")

model = network.NN(loss='cross_entropy')
model.add_layer(h1)
model.add_layer(h2)
model.add_layer(h3)
model.add_layer(o1)
model.summary()

print("")
model.fit(X_train, y_train, max_iter=max_iter, 
          lr=lr, epsilon=eps, b_sz = b_sz,
          X_val=X_val, Y_val=y_val,
          print_interval=print_interval)
iteraction_log = network.get_iteration_log()

Y_ = np.array(model.predict(X_val)).argmax(axis=-1)
reload(custom_scores)
custom_scores.evaluate_multiclass(y_val=y_val.argmax(axis=-1), y_pred=Y_)
iteraction_log.index = iteraction_log.it
iteraction_log.error_train.plot()
iteraction_log.error_val.plot()